This notebook weaves together the haplotype scaffold at biallelic sites phased via shapeit2 with the multiallelic and other extra (e.g., non-PASS) variants phased via mvncall.

In [1]:
%run setup.ipynb

In [2]:
# haplotype scaffold
callset_phased = phase2_ar1.callset_phased
gt_phased = callset_phased['2L']['calldata/genotype']
gt_phased, gt_phased.chunks

(<HDF5 dataset "genotype": shape (8906423, 1164, 2), type "|i1">,
 (131072, 60, 2))

In [3]:
pos_phased = allel.SortedIndex(callset_phased['2L']['variants/POS'])
pos_phased.shape, gt_phased.shape

((8906423,), (8906423, 1164, 2))

In [4]:
# define region we're going to analyse
loc_region = pos_phased.locate_range(0, 6000000)
loc_region

slice(0, 390585, None)

In [5]:
# locate wild-caught samples
df_samples = phase2_ar1.df_samples
df_samples.head()

,src_code,population,country,region,contributor,contact,year,m_s,sex,n_sequences,mean_coverage
ox_code,,,,,,,,,,,
AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo_Praso,David Weetman,NaN,2012.0,M,F,95033368,30.99
AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo_Praso,David Weetman,NaN,2012.0,M,F,95843804,31.70
AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,David Weetman,NaN,2012.0,M,F,107420666,35.65
AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,David Weetman,NaN,2012.0,M,F,95993752,29.46
AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,David Weetman,NaN,2012.0,M,F,103044262,33.67


In [6]:
len(df_samples)

1142

In [7]:
callset_samples = callset_phased['2L']['samples'][:].astype('U').tolist()
callset_samples[:5], len(callset_samples)

(['AA0040-C', 'AA0041-C', 'AA0042-C', 'AA0043-C', 'AA0044-C'], 1164)

In [8]:
sample_indices = [callset_samples.index(s) for s in df_samples.index.values]
sample_indices[:5]

[0, 1, 2, 3, 4]

In [9]:
np.all(np.diff(sample_indices) == 1)

True

In [10]:
# extract data for region, remove colony parents 
pos_phased_region = pos_phased[loc_region]
gt_phased_region = gt_phased[loc_region].astype('i1').take(sample_indices, axis=1)
pos_phased_region.shape, gt_phased_region.shape, gt_phased_region.dtype

((390585,), (390585, 1142, 2), dtype('int8'))

In [11]:
# TODO

# load mvn haplotypes
# callset_extras = np.load('../data/phasing_extra_phase1.mvncall.200.npz')
# pos_extras = callset_extras['variants']['POS']
# gt_extras = callset_extras['calldata']['genotype']
# pos_extras.shape, gt_extras.shape

In [12]:
# TODO

# # concatenate
# gt_combined = np.concatenate([gt_phased_region, gt_extras], axis=0)
# pos_combined = np.concatenate([pos_phased_region, pos_extras], axis=0)

# # sort by position
# idx_sorted = np.argsort(pos_combined)
# gt_combined = gt_combined[idx_sorted]
# pos_combined = pos_combined[idx_sorted]

# TODO remove this hack
gt_combined = gt_phased_region
pos_combined = pos_phased_region

In [13]:
# obtain data from unphased callset - only needed for variant annotations
callset_snpeff = phase2_ar1.callset_snpeff_agamp42
pos_all = allel.SortedIndex(callset_snpeff['2L']['variants/POS'])
ann_all = callset_snpeff['2L']['variants/ANN'][:][['Annotation', 'HGVS_p', 'HGVS_c']]
ann_all

array([(b'intergenic_region', b'.', b'.'),
       (b'intergenic_region', b'.', b'.'),
       (b'intergenic_region', b'.', b'.'), ...,
       (b'intergenic_region', b'.', b'.'),
       (b'intergenic_region', b'.', b'.'),
       (b'intergenic_region', b'.', b'.')], 
      dtype=[('Annotation', 'S34'), ('HGVS_p', 'S14'), ('HGVS_c', 'S12')])

In [14]:
# locate the intersection with unphased callset - needed to tie in annotations
loc1, _ = pos_all.locate_intersection(pos_combined)
np.count_nonzero(loc1)

390585

In [15]:
# extract annotations for the phased variants
ann_combined = ann_all[loc1]
ann_combined

array([(b'intergenic_region', b'.', b'.'),
       (b'intergenic_region', b'.', b'.'),
       (b'intergenic_region', b'.', b'.'), ...,
       (b'upstream_gene_variant', b'.', b'n.-9A>T'),
       (b'upstream_gene_variant', b'.', b'n.-9G>T'),
       (b'upstream_gene_variant', b'.', b'n.-9C>G')], 
      dtype=[('Annotation', 'S34'), ('HGVS_p', 'S14'), ('HGVS_c', 'S12')])

In [16]:
# save
haps_combined = allel.GenotypeArray(gt_combined).to_haplotypes()
np.savez_compressed('../../data/phase2/haps.npz', haplotypes=haps_combined, POS=pos_combined, ANN=ann_combined)

In [17]:
haps_combined.nbytes

892096140

In [18]:
!ls -lh ../../data/phase2/haps.npz

-rw-r--r-- 1 aliman aliman 17M Nov  3 12:50 ../../data/phase2/haps.npz
